In [1]:
import stk
import networkx as nx
import os
from rdkit.Chem import AllChem as rdkit
import py3Dmol

In [2]:
def show_stk_mol(stk_mol):
    data = rdkit.MolToMolBlock(stk_mol.to_rdkit_mol())
    p = py3Dmol.view(
        data=data,
        style={'stick':{'colorscheme':'cyanCarbon'}}, 
        width=400,
        height=400,
    )
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    p.show()

# Use NetworkX to split molecules.

In [3]:
def get_disconnected_components(molecule):

    # Produce a graph from the molecule that does not include 
    # edges where the bonds to be optimized are.
    mol_graph = nx.Graph()
    for atom in molecule.get_atoms():
        mol_graph.add_node(atom.get_id())

    # Add edges.
    for bond in molecule.get_bonds():
        pair_ids = (
            bond.get_atom1().get_id(), bond.get_atom2().get_id()
        )
        mol_graph.add_edge(*pair_ids)

    # Get atom ids in disconnected subgraphs.
    components = {}
    for c in nx.connected_components(mol_graph):
        c_ids = sorted(c)
        molecule.write('temp_mol.mol', atom_ids=c_ids)
        num_atoms = len(c_ids)
        newbb = stk.BuildingBlock.init_from_file('temp_mol.mol')
        os.system('rm temp_mol.mol')

        components[num_atoms] = newbb

    return components

In [4]:
def extract_host(molecule):
    components = get_disconnected_components(molecule)
    return components[max(components.keys())]

def extract_guest(molecule):
    components = get_disconnected_components(molecule)
    return components[min(components.keys())]

# Files

In [5]:
host_with_g_file = 'orig_hostguest.mol'

In [6]:
new_guest_file = 'new_guest.mol'

# Given host-guest, replace guest from .mol

In [7]:
# Load in host-guest.
host_with_guest = stk.BuildingBlock.init_from_file(
    path=host_with_g_file,
)

# Load in new guest.
new_guest = stk.BuildingBlock.init_from_file(new_guest_file)

In [9]:
show_stk_mol(host_with_guest)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [10]:
show_stk_mol(new_guest)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [11]:
# Split host and guest, assuming host has more atoms than guest.
old_host = extract_host(host_with_guest)
old_guest = extract_guest(host_with_guest)

In [12]:
show_stk_mol(old_host)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [13]:
show_stk_mol(old_guest)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## You could also keep this for analysis!

In [14]:
# Build new host-guest structure, with Spindry optimiser to 
# do some conformer searching.
new_host_with_guest = stk.ConstructedMolecule(
    topology_graph=stk.host_guest.Complex(
        host=stk.BuildingBlock.init_from_molecule(old_host),
        guests=(stk.host_guest.Guest(new_guest), ),
        optimizer=stk.Spinner(),
    ),
)

In [15]:
show_stk_mol(new_host_with_guest)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Given host-guest, replace guest from smiles

In [16]:
# Load in host-guest.
host_with_guest = stk.BuildingBlock.init_from_file(
    path=host_with_g_file,
)

# Load in new guest.
new_guest = stk.BuildingBlock('CN1C=NC2=C1C(=O)N(C(=O)N2C)C')

In [17]:
show_stk_mol(host_with_guest)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [18]:
show_stk_mol(new_guest)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [19]:
# Split host and guest, assuming host has more atoms than guest.
old_host = extract_host(host_with_guest)

In [20]:
show_stk_mol(old_host)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [25]:
# Build new host-guest structure, with Spindry optimiser to 
# do some conformer searching.
new_host_with_guest = stk.ConstructedMolecule(
    topology_graph=stk.host_guest.Complex(
        host=stk.BuildingBlock.init_from_molecule(old_host),
        guests=(stk.host_guest.Guest(new_guest), ),
        optimizer=stk.Spinner(num_conformers=5),
    ),
)

In [26]:
show_stk_mol(new_host_with_guest)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Given host-guest, replace host from constructed molecule

In [27]:
# Load in host-guest.
host_with_guest = stk.BuildingBlock.init_from_file(
    path=host_with_g_file,
)

In [28]:
# Define new host.
# Produce a Fe+2 atom with 6 functional groups.
iron_atom = stk.BuildingBlock(
    smiles='[Fe+2]',
    functional_groups=(
        stk.SingleAtom(stk.Fe(0, charge=2))
        for i in range(6)
    ),
    position_matrix=[[0, 0, 0]],
)

# Define coordinating ligand with dummy bromine groups and
# metal coordinating functional groups.
bb2 = stk.BuildingBlock(
    smiles='C1=NC(C=NBr)=CC=C1',
    functional_groups=[
        stk.SmartsFunctionalGroupFactory(
            smarts='[#6]~[#7X2]~[#35]',
            bonders=(1, ),
            deleters=(),
        ),
        stk.SmartsFunctionalGroupFactory(
            smarts='[#6]~[#7X2]~[#6]',
            bonders=(1, ),
            deleters=(),
        ),
    ],
)

# Build iron complex with delta stereochemistry.
iron_oct_delta = stk.ConstructedMolecule(
    topology_graph=stk.metal_complex.OctahedralDelta(
        metals=iron_atom,
        ligands=bb2,
        optimizer=stk.MCHammer(),
    ),
)
# Assign Bromo functional groups to the metal complex.
iron_oct_delta = stk.BuildingBlock.init_from_molecule(
    molecule=iron_oct_delta,
    functional_groups=[stk.BromoFactory()],
)

# Define spacer building block.
bb3 = stk.BuildingBlock(
    smiles=(
        'C1=CC(C2=CC=C(Br)C=C2)=C'
        'C=C1Br'
    ),
    functional_groups=[stk.BromoFactory()],
)

# Build an M4L6 Tetrahedron with a spacer.
new_host = stk.ConstructedMolecule(
    topology_graph=stk.cage.M4L6TetrahedronSpacer(
        building_blocks=(
            iron_oct_delta,
            bb3,
        ),
        optimizer=stk.MCHammer(),
    ),
)


In [29]:
show_stk_mol(host_with_guest)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [30]:
show_stk_mol(new_host)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [31]:
# Split host and guest, assuming host has more atoms than guest.
old_guest = extract_guest(host_with_guest)

In [32]:
show_stk_mol(old_guest)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [33]:
# Build new host-guest structure, with Spindry optimiser to 
# do some conformer searching.
new_host_with_guest = stk.ConstructedMolecule(
    topology_graph=stk.host_guest.Complex(
        host=stk.BuildingBlock.init_from_molecule(new_host),
        guests=(stk.host_guest.Guest(old_guest), ),
        optimizer=stk.Spinner(),
    ),
)

In [34]:
show_stk_mol(new_host_with_guest)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol